In [2]:
import pandas as pd
import schools
df = schools.load_demographics()

df.head()

,dbn,school_name,year,total_enrollment,grade_3k_pk_half_day_full,grade_k,grade_1,grade_2,grade_3,grade_4,...,clean_name,short_name,non_white,non_white_1,black_hispanic,black_hispanic_1,white_asian,white_asian_1,non_white_asian,non_white_asian_1
0,01M015,P.S. 015 Roberto Clemente,2015,176,14,32,33,39,23,17,...,roberto clemente,PS 15,174,0.988636,162,0.920455,11,0.062500,165,0.937500
1,01M015,P.S. 015 Roberto Clemente,2016,178,17,28,33,27,31,24,...,roberto clemente,PS 15,174,0.977528,156,0.876404,18,0.101124,160,0.898876
2,01M015,P.S. 015 Roberto Clemente,2017,190,17,28,32,33,23,31,...,roberto clemente,PS 15,184,0.968421,162,0.852632,26,0.136842,164,0.863158
3,01M015,P.S. 015 Roberto Clemente,2018,174,13,20,33,30,30,20,...,roberto clemente,PS 15,168,0.965517,143,0.821839,30,0.172414,144,0.827586
4,01M015,P.S. 015 Roberto Clemente,2019,190,14,29,28,38,33,29,...,roberto clemente,PS 15,181,0.952632,152,0.800000,36,0.189474,154,0.810526


In [3]:
MATH_url = "https://data.cityofnewyork.us/resource/m27t-ht3h.csv?$limit=10000000"
mdf = pd.read_csv(MATH_url)
mdf.head()

,dbn,school_name,grade,year,category,number_tested,mean_scale_score,level_1,level_1_1,level_2,level_2_1,level_3,level_3_1,level_4,level_4_1,level_3_4,level_3_4_1
0,01M015,P.S. 015 Roberto Clemente,3,2013,All Students,27,278,16,59.3,11,40.7,0,0.0,0,0.0,0,0.0
1,01M015,P.S. 015 Roberto Clemente,3,2014,All Students,18,286,6,33.3,9,50.0,2,11.1,1,5.6,3,16.7
2,01M015,P.S. 015 Roberto Clemente,3,2015,All Students,17,280,10,58.8,4,23.5,2,11.8,1,5.9,3,17.6
3,01M015,P.S. 015 Roberto Clemente,3,2016,All Students,21,275,13,61.9,4,19.0,4,19.0,0,0.0,4,19.0
4,01M015,P.S. 015 Roberto Clemente,3,2017,All Students,29,302,8,27.6,9,31.0,7,24.1,5,17.2,12,41.4


In [7]:
# mdf.set_index(["dbn", "year"])
# df.set_index(["dbn", "year"])

mdf["category"].unique()
mdf.columns

Index(['dbn', 'school_name', 'grade', 'year', 'category', 'number_tested',
       'mean_scale_score', 'level_1', 'level_1_1', 'level_2', 'level_2_1',
       'level_3', 'level_3_1', 'level_4', 'level_4_1', 'level_3_4',
       'level_3_4_1'],
      dtype='object')

In [4]:
del mdf["school_name"]

mdf["category"] = mdf["category"].apply(lambda cat: cat.lower())
mdf.loc[mdf['category'] == "all students", 'category'] = "all"
mdf.loc[mdf['category'] == "all students", 'category'] = "all"
mdf.loc[mdf['category'] == "never ell", 'category'] = "never_ell"
mdf.loc[mdf['category'] == "ever ell", 'category'] = "ever_ell"
mdf.loc[mdf['category'] == "current ell", 'category'] = "current_ell"
mdf.loc[mdf['category'] == "swd", 'category'] = "swd"
mdf.loc[mdf['category'] == "not swd", 'category'] = "not_swd"
mdf.loc[mdf['category'] == "swd", 'category'] = "swd"
mdf.loc[mdf['category'] == "econ disadv", 'category'] = "econ_disadv"
mdf.loc[mdf['category'] == "not econ disadv", 'category'] = "not_econ_disadv"
mdf.loc[mdf['grade'] == "all grades", 'grade'] = "all"

mdf.loc[mdf['grade'] == "All Grades", 'grade'] = "all"

In [5]:
from itertools import chain
def make_test_cols(test_df):
    grades = test_df.grade.unique()
    cats = test_df.category.unique()
    combos = [[(cat, grade) for grade in grades] for cat in cats]
    combos = list(chain.from_iterable(combos))
    return combos

combos = make_test_cols(mdf)



In [1]:

# combined = df.copy()


def rows_to_cols(cat, grade, test_df, prefix="math"):
    """
        get the test result rows for a category and grade
        and make them columns in a new dataframe
    """
    test_cols = ['number_tested',
        'mean_scale_score',
        'level_1',
        'level_1_1',
        'level_2',
        'level_2_1',
        'level_3',
        'level_3_1',
        'level_4',
        'level_4_1',
        'level_3_4',
        'level_3_4_1']
    temp_df = test_df.query(f"category == '{cat}' and grade == '{grade}'")
    new_cols = [f"{prefix}_{cat}_grade_{grade}_{col}" for col in test_cols]
    rename = dict(zip(test_cols, new_cols))
    temp_df = temp_df.rename(columns = rename)
    temp_df = temp_df.drop(["grade", "category"], axis=1)
    temp_df.set_index(["dbn", "year"])
    return temp_df



math_df = [rows_to_cols(cat, grade, mdf, "math") for cat, grade in combos]





combined = df.copy()

# combined = combined.combine_first(math_df[0])


# combined.set_index(["dbn", "year"])

for t in math_df:
    combined = combined.merge(t,on=["dbn", "year"], how="left")

combined.tail()







NameError: name 'combos' is not defined